In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
import cvzone # only to create hand outline inlay
import random
#import tqdm # only for progress bar

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hand = mp.solutions.hands
mp_face_detection = mp.solutions.face_detection

In [3]:
idealhandLndmrk = {5:{'x':0.3498418629169464, 'y':0.47491520643234253},17:{'x':0.22911912202835083, 'y':0.5145605802536011}, 0:{'x':0.2968618869781494, 'y':0.784920334815979}}
idealfaceLndmrk = {'xmin': 0.5979199409484863, 'ymin': 0.3973429799079895, 'width': 0.1899573802947998, 'height': 0.3376706838607788}

In [4]:
# Function to check the number being shown

def checkSign(handLndmrk):
    txt = 'Not recognized'          
    if (handLndmrk[4].x > handLndmrk[2].x) & (handLndmrk[20].y < handLndmrk[18].y) & (handLndmrk[16].y < handLndmrk[14].y) & (handLndmrk[12].y < handLndmrk[10].y) & (handLndmrk[8].y < handLndmrk[6].y):
                    txt = 'Five'
    elif (handLndmrk[20].y < handLndmrk[18].y) & (handLndmrk[16].y < handLndmrk[14].y) & (handLndmrk[12].y < handLndmrk[10].y) & (handLndmrk[8].y < handLndmrk[6].y) & (handLndmrk[4].x < handLndmrk[2].x):
                    txt = 'Four'
    elif (handLndmrk[16].y < handLndmrk[14].y) & (handLndmrk[12].y < handLndmrk[10].y) & (handLndmrk[8].y < handLndmrk[6].y) & (handLndmrk[20].y > handLndmrk[18].y) & (handLndmrk[4].x < handLndmrk[2].x):
                    txt = 'Three'
    elif (handLndmrk[12].y < handLndmrk[10].y) & (handLndmrk[8].y < handLndmrk[6].y) & (handLndmrk[16].y > handLndmrk[14].y) & (handLndmrk[20].y > handLndmrk[18].y) & (handLndmrk[4].x < handLndmrk[2].x):  
                    txt = 'Two'
    elif (handLndmrk[8].y < handLndmrk[6].y) & (handLndmrk[12].y > handLndmrk[10].y) & (handLndmrk[16].y > handLndmrk[14].y) & (handLndmrk[20].y > handLndmrk[18].y) & (handLndmrk[4].x < handLndmrk[2].x):
                    txt = 'One'
    elif (handLndmrk[8].y > handLndmrk[6].y) & (handLndmrk[12].y > handLndmrk[10].y) & (handLndmrk[16].y > handLndmrk[14].y) & (handLndmrk[20].y > handLndmrk[18].y) & (handLndmrk[4].x < handLndmrk[2].x):
                    txt = 'Zero'
                    
    return txt

In [5]:
# function to check whether the user has placed their hand on the indicated section. Any changes to the dimensions in 
# showBox function must also reflect here.

def checkValidity(handLndmrk, faceCoord, ShowBox = False, toleranceFace = 150, toleranceHand = 60):
        
    if ShowBox == True:
        showBox(img, toleranceHand, toleranceFace)
    if faceCheck(faceCoord, toleranceFace):
        if handCheck(handLndmrk, toleranceHand):
            return True
    return False


def faceCheck(faceCoord, toleranceFace):
    if ((faceCoord.xmin*img.shape[1]) > (round(idealfaceLndmrk['xmin']*img.shape[1])-toleranceFace)) & ((faceCoord.ymin*img.shape[0]) > (round(idealfaceLndmrk['ymin']*img.shape[0])-toleranceFace)):
        if (((faceCoord.xmin+faceCoord.width)*img.shape[1])<(round((idealfaceLndmrk['xmin']+idealfaceLndmrk['width'])*img.shape[1])+toleranceFace)) & (((faceCoord.ymin+faceCoord.height)*img.shape[0])<(round((idealfaceLndmrk['ymin']+idealfaceLndmrk['height'])*img.shape[0])+toleranceFace)):
            return True
    return False
    
def handCheck(handLndmrk, toleranceHand):
    if (abs((handLndmrk[5].x*img.shape[1])-round(idealhandLndmrk[5]['x']*img.shape[1])) < toleranceHand) & (abs((handLndmrk[5].y*img.shape[0])-round(idealhandLndmrk[5]['y']*img.shape[0])) < toleranceHand):
        if (abs((handLndmrk[17].x*img.shape[1])-round(idealhandLndmrk[17]['x']*img.shape[1])) < toleranceHand) & (abs((handLndmrk[17].y*img.shape[0])-round(idealhandLndmrk[17]['y']*img.shape[0])) < toleranceHand):
            if (abs((handLndmrk[0].x*img.shape[1])-round(idealhandLndmrk[0]['x']*img.shape[1])) < toleranceHand) & (abs((handLndmrk[0].y*img.shape[0])-round(idealhandLndmrk[0]['y']*img.shape[0])) < toleranceHand):
                return True
    return False

In [6]:
# Function to show the detection boxes used to check validity. Any changes to the dimensions in 
# checkValidity function must also reflect here.

def showBox(img, toleranceHand, toleranceFace):
    #face box
    cv2.rectangle(img, 
                  (round(idealfaceLndmrk['xmin']*img.shape[1])-toleranceFace, round(idealfaceLndmrk['ymin']*img.shape[0])-toleranceFace),
                  (round((idealfaceLndmrk['xmin']+idealfaceLndmrk['width'])*img.shape[1])+toleranceFace, 
                   round((idealfaceLndmrk['ymin']+idealfaceLndmrk['height'])*img.shape[0])+toleranceFace),
                  (255,141,47), 2)
    
    #partition based validation
#     cv2.line(img, (round(img.shape[1]*0.5), 0), (round(img.shape[1]*0.5), img.shape[0]), (255,141,47), 2)
    
    # point5
    cv2.rectangle(img, (round(idealhandLndmrk[5]['x']*img.shape[1])-toleranceHand, round(idealhandLndmrk[5]['y']*img.shape[0])-toleranceHand), (round(idealhandLndmrk[5]['x']*img.shape[1])+toleranceHand, round(idealhandLndmrk[5]['y']*img.shape[0])+toleranceHand), (255, 141, 47), 2)
    cv2.circle(img, (round(idealhandLndmrk[5]['x']*img.shape[1]),round(idealhandLndmrk[5]['y']*img.shape[0])), radius=1, color=(255, 0, 0), thickness=3)
    # point17
    cv2.rectangle(img, (round(idealhandLndmrk[17]['x']*img.shape[1])-toleranceHand, round(idealhandLndmrk[17]['y']*img.shape[0])-toleranceHand), (round(idealhandLndmrk[17]['x']*img.shape[1])+toleranceHand, round(idealhandLndmrk[17]['y']*img.shape[0])+toleranceHand), (255, 141, 47), 2)
    cv2.circle(img, (round(idealhandLndmrk[17]['x']*img.shape[1]),round(idealhandLndmrk[17]['y']*img.shape[0])), radius=1, color=(255, 0, 0), thickness=3)
    # point0
    cv2.rectangle(img, (round(idealhandLndmrk[0]['x']*img.shape[1])-toleranceHand, round(idealhandLndmrk[0]['y']*img.shape[0])-toleranceHand), (round(idealhandLndmrk[0]['x']*img.shape[1])+toleranceHand, round(idealhandLndmrk[0]['y']*img.shape[0])+toleranceHand), (255, 141, 47), 2)
    cv2.circle(img, (round(idealhandLndmrk[0]['x']*img.shape[1]),round(idealhandLndmrk[0]['y']*img.shape[0])), radius=1, color=(255, 0, 0), thickness=3)
    
    return True



In [7]:
def game(handLndmrk, l):
    global counter
    t = checkSign(handLndmrk)
    if t == l:
        counter+=1
    elif (t!=l) & (counter>0) :
        counter-=1
    return counter,t
        
def createnNum():
    l = random.choice(['Zero', 'One', 'Two', 'Three', 'Four', 'Five'])
    return l
        



In [8]:
# image used to overlay the hand outline

overlay = cv2.imread(r'C:\Users\USER\Desktop\machineLearningEx\images\hand4-res.png', cv2.IMREAD_UNCHANGED)


In [11]:
# main code to open camera and perform detection.

capture = cv2.VideoCapture(0)
capture.set(3, 1280)
capture.set(4, 960)

with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection: 
    with mp_hand.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands=1) as hands:
            flag = False
            counter = 0
            l=''
            while True:
                ret, img = capture.read()
                result = hands.process(img)
                resultface = face_detection.process(img)
                txt = None
                txt2 = 'Place your right hand on the red box and your face on the blue box'
                if resultface.detections:
                      for detection in resultface.detections:
                            faceCoord = detection.location_data.relative_bounding_box
#                             mp_drawing.draw_detection(img, detection)
                
                #Remove this once finished
                cv2.rectangle(img, 
                  (round(idealfaceLndmrk['xmin']*img.shape[1])-100, round(idealfaceLndmrk['ymin']*img.shape[0])-100),
                  (round((idealfaceLndmrk['xmin']+idealfaceLndmrk['width'])*img.shape[1])+100, 
                   round((idealfaceLndmrk['ymin']+idealfaceLndmrk['height'])*img.shape[0])+100),
                  (255,141,47), 2)
                
                if result.multi_hand_landmarks:
                    for num,hand in enumerate(result.multi_hand_landmarks):
                        mp_drawing.draw_landmarks(img, hand, mp_hand.HAND_CONNECTIONS)
                        handLndmrk = hand.landmark
                        
                    if checkValidity(handLndmrk, faceCoord, ShowBox = True):
                        if flag == False:
                            l = createnNum()
                            flag = True
                        txt2 = 'Detection is live, Show me the number: '+ l
                        counter, txt = game(handLndmrk, l)
                        if counter == 100:
                            flag = False
                            counter = 0

                #showBox(img)
#                 img = cv2.flip(img, flipCode = 90) 
                if l == txt:
                    cv2.putText(img, txt,(20,120),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3, cv2.LINE_AA)
                if l != txt:
                    cv2.putText(img, txt,(20,120),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA)

                cv2.putText(img, txt2,(20,50),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 141, 47), 2, cv2.LINE_AA)

#                 img = cvzone.overlayPNG(img, overlay, [450,80])
                
                cv2.rectangle(img,(1070,80),(630,600),(0,0,255), 2)
                
                


                cv2.imshow('Web cam:', img)

                if cv2.waitKey(5) & 0xFF == ord('q'):
                     break
                    
capture.release()                    
cv2.destroyAllWindows()


In [ ]:
capture.release()
cv2.destroyAllWindows()

In [ ]:
detection.location_data.relative_bounding_box

In [ ]:
# Ideal points taken

# handLndmrk[5]
# x: 0.3498418629169464
# y: 0.47491520643234253
# z: -0.025790967047214508
# handLndmrk[17]
# x: 0.22911912202835083
# y: 0.5145605802536011
# z: -0.030962718650698662
# handLndmrk[0]
# x: 0.2968618869781494
# y: 0.784920334815979
# z: 5.765244281974446e-07

# img.shape
# (720, 1280, 3)